In [1]:
import cv2
import dlib
import numpy as np
import mediapipe as mp
from mtcnn import MTCNN

/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [26]:
detector_dlib = dlib.get_frontal_face_detector()
predictor_dlib = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [27]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

In [28]:
detector_mtcnn = MTCNN()

In [29]:
# Функции для определения того, открыты ли глаза, и улыбается ли человек

def eye_aspect_ratio(pts): # вертикальное расстояние между веками делим на ширину глаза
    A = np.linalg.norm(pts[1]-pts[5])
    B = np.linalg.norm(pts[2]-pts[4])
    C = np.linalg.norm(pts[0]-pts[3])
    return (A + B) / (2.0 * C)

def is_smiling_dlib(coords): # ширина рта + подняты ли уголки
    mouth = coords[48:60]
    mouth_w = np.linalg.norm(mouth[0]-mouth[6])
    mouth_h = np.linalg.norm(mouth[3]-mouth[9])
    interocular = np.linalg.norm(coords[36]-coords[45])  # расстояние между глазами

    smile_ratio = mouth_w / interocular
    corners_up = (mouth[0][1] < mouth[3][1]) and (mouth[6][1] < mouth[9][1])

    return (smile_ratio > 0.45) and corners_up

In [30]:
# Dlib
def analyze_dlib(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector_dlib(gray, 1)
    out = []
    for f in faces:
        shape = predictor_dlib(gray, f)
        coords = np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)])
        left_eye = coords[36:42]; right_eye = coords[42:48]
        ear_l = eye_aspect_ratio(left_eye); ear_r = eye_aspect_ratio(right_eye)
        eyes_open = (ear_l > 0.22) and (ear_r > 0.22)
        smiling = is_smiling_dlib(coords)
        out.append(dict(smile=smiling, eyes_open=eyes_open))
    return out

In [31]:
# MediaPipe
mp_face_mesh = mp.solutions.face_mesh

def analyze_mp(img):
    mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    res = mesh.process(rgb)
    out = []
    if res.multi_face_landmarks:
        for lm in res.multi_face_landmarks:
            h,w = img.shape[:2]
            def p(i): return np.array([lm.landmark[i].x*w, lm.landmark[i].y*h])
            mouth_w = np.linalg.norm(p(61)-p(291))
            mouth_h = np.linalg.norm(p(13)-p(14))
            interocular = np.linalg.norm(p(33)-p(263))
            smile_ratio = mouth_w / interocular
            corners_up = (p(61)[1] < p(13)[1]) and (p(291)[1] < p(14)[1])
            smile = (smile_ratio > 0.45) and corners_up
            eye_l_open = np.linalg.norm(p(159)-p(145)) / np.linalg.norm(p(33)-p(133)) > 0.22
            eye_r_open = np.linalg.norm(p(386)-p(374)) / np.linalg.norm(p(263)-p(362)) > 0.22
            out.append(dict(smile=smile, eyes_open=(eye_l_open and eye_r_open)))
    mesh.close()
    return out

In [32]:
# OpenCV каскады
def analyze_cv(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    out = []
    for (x,y,w,h) in faces:
        roi = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi, 1.1, 10)
        smiles = smile_cascade.detectMultiScale(roi, 1.7, 20)
        eyes_open = len(eyes) >= 2
        # каскад для улыбки часто шумный, поэтому проверим ширину рта относительно лица
        smiling = len(smiles) > 0 and (smiles[0][2] / w > 0.3)
        out.append(dict(smile=smiling, eyes_open=eyes_open))
    return out


In [33]:
# MTCNN
def analyze_mtcnn(img):
    res = detector_mtcnn.detect_faces(img)
    out = []
    for r in res:
        k = r['keypoints']
        mouth_w = np.linalg.norm(np.array(k['mouth_right']) - np.array(k['mouth_left']))
        interocular = np.linalg.norm(np.array(k['left_eye']) - np.array(k['right_eye']))
        smile_ratio = mouth_w / interocular
        smiling = smile_ratio > 0.45
        out.append(dict(smile=smiling, eyes_open=True))  # глаза MTCNN не различает
    return out

In [46]:
from google.colab import files

uploaded = files.upload()   # откроется окно выбора файла

Saving taylor_swift.jpg to taylor_swift.jpg


In [35]:
# человек улыбается с открытыми глазами

img = cv2.imread("13.jpg")  # замените на свой файл

print("Dlib:", analyze_dlib(img))
print("MediaPipe:", analyze_mp(img))
print("OpenCV:", analyze_cv(img))
print("MTCNN:", analyze_mtcnn(img))

Dlib: [{'smile': False, 'eyes_open': True}]
MediaPipe: [{'smile': True, 'eyes_open': True}]
OpenCV: [{'smile': True, 'eyes_open': True}]
MTCNN: [{'smile': True, 'eyes_open': True}]


Dlib не определил улыбку (возможно, слишком строгий порог), всё остальное верно.

In [45]:
# человек улыбается с закрытыми глазами

img = cv2.imread("smile_and_closed.jpg")  # замените на свой файл

print("Dlib:", analyze_dlib(img))
print("MediaPipe:", analyze_mp(img))
print("OpenCV:", analyze_cv(img))
print("MTCNN:", analyze_mtcnn(img))

Dlib: [{'smile': False, 'eyes_open': False}]
MediaPipe: [{'smile': False, 'eyes_open': False}]
OpenCV: []
MTCNN: []


Все методы не распознали улыбку, хотя она была. Это значит, что критерий улыбки слишком жёсткий, особенно при закрытых глазах (возможно, из-за того, что геометрия лица меняется). OpenCV/MTCNN вообще не нашли лицо (мешают закрытые глаза может).

In [39]:
# человек не улыбается с закрытыми глазами

img = cv2.imread("no_smile_and_closed.jpg")  # замените на свой файл

print("Dlib:", analyze_dlib(img))
print("MediaPipe:", analyze_mp(img))
print("OpenCV:", analyze_cv(img))
print("MTCNN:", analyze_mtcnn(img))

Dlib: [{'smile': False, 'eyes_open': False}]
MediaPipe: [{'smile': False, 'eyes_open': False}]
OpenCV: [{'smile': True, 'eyes_open': True}]
MTCNN: [{'smile': True, 'eyes_open': True}]


Dlib и MediaPipe хорошо справились, OpenCV и MTCNN в точности наоборот.

In [41]:
# человек не улыбается с открытыми глазами

img = cv2.imread("test_no_make_up.jpg")  # замените на свой файл

print("Dlib:", analyze_dlib(img))
print("MediaPipe:", analyze_mp(img))
print("OpenCV:", analyze_cv(img))
print("MTCNN:", analyze_mtcnn(img))

Dlib: [{'smile': False, 'eyes_open': True}]
MediaPipe: [{'smile': False, 'eyes_open': True}]
OpenCV: [{'smile': False, 'eyes_open': True}]
MTCNN: [{'smile': True, 'eyes_open': True}]


MTCNN ошибся в улыбке, но насчёт глаз ему не доверяем тоже, он всегда говорит True.

Общие выводы небольшого эксперимента:

Dlib: надёжно определяет глаза, но не видит улыбку (необходима коррекция алгоритма определения).

MediaPipe: чаще всего совпадал с реальностью.

OpenCV: каскады оказалсиь нестабильными (ложные улыбки и даже не всегда находит лицо вообще).

MTCNN: тоже плохой за счёт маленкього количества точек, что затрудняет определение улыбки и проверки глаз.